<a href="https://colab.research.google.com/github/4graf/Data_Processing/blob/main/Parser%20YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***JSON***

Создание файла *youtube.json*, содержащий **каналы с видео**, а также **любимые видео**:

In [ ]:
import json

channels = {'Aim To Head Mix' : ['Dark Techno / EBM / Industrial Bass Mix \'HAVOC vol.2\''], 'Davie504' : ['BASSta\'s Paradise (Official Video)', 'These Russian Bassists Need to be STOPPED', 'so Snoop Dogg sent me this..'], 'Chillhop Music' : ['lofi hip hop radio - beats to study/relax to ']}
favourite_videos = ['BASSta\'s Paradise (Official Video)', 'These Russian Bassists Need to be STOPPED', 'Dark Techno / EBM / Industrial Bass Mix \'HAVOC vol.2\'']

youtube = {'channels' : channels, 'favourite_videos' : favourite_videos}

with open('youtube.json', 'w') as f:
  json.dump(youtube, f, indent=4)

Чтение файла *youtube.json* и выбор канала *Davie504* с проверкой: являются ли его видео **любимыми**:

In [ ]:
import json

with open('youtube.json', 'r') as f:
  youtube = json.load(f)

# print('Содержимое youtube.json:', youtube)
print('Содержимое youtube.json:', json.dumps(youtube, indent=4))

davie504_videos = youtube['channels']['Davie504']
print('\nВидео Davie504:', davie504_videos)

print('\nЛюбимые видео с канала Davie504:')
for vid in davie504_videos:
  if vid in youtube['favourite_videos']:
    print(vid)

Содержимое youtube.json: {
    "channels": {
        "Aim To Head Mix": [
            "Dark Techno / EBM / Industrial Bass Mix 'HAVOC vol.2'"
        ],
        "Davie504": [
            "BASSta's Paradise (Official Video)",
            "These Russian Bassists Need to be STOPPED",
            "so Snoop Dogg sent me this.."
        ],
        "Chillhop Music": [
            "lofi hip hop radio - beats to study/relax to "
        ]
    },
    "favourite_videos": [
        "BASSta's Paradise (Official Video)",
        "These Russian Bassists Need to be STOPPED",
        "Dark Techno / EBM / Industrial Bass Mix 'HAVOC vol.2'"
    ]
}

Видео Davie504: ["BASSta's Paradise (Official Video)", 'These Russian Bassists Need to be STOPPED', 'so Snoop Dogg sent me this..']

Любимые видео с канала Davie504:
BASSta's Paradise (Official Video)
These Russian Bassists Need to be STOPPED


# ***Парсер с API Youtube***

Для получения данных с youtube разработчиками google был создан специальный **апи** для удобного парсинга информации. Для работы с апи необходим **личный ключ разработчика**:

In [ ]:
import os
import googleapiclient.discovery
import json

# Ключ апи, полученный на https://developers.google.com/youtube/v3
API_KEY = ''

def get_videos_api(channel_name, count_video=5):
    # Отключение OAuthlib's HTTPS верификации при локальном запуске
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
 
    youtube = googleapiclient.discovery.build(api_service_name, api_version, 
                                              developerKey=API_KEY)

    request_channel = youtube.channels().list(part="snippet,contentDetails,statistics", 
        forUsername=channel_name)

    request_channel = request_channel.execute()
    channel_items = request_channel['items'] 
    channel_id = channel_items[0]['id']
    channel_upload_playlist = channel_items[0]['contentDetails']['relatedPlaylists']['uploads']

    # print(channel_upload_playlist)
    # print(json.dumps(request_channel, indent=4))

    request_video = youtube.playlistItems().list(
        part="snippet",
        playlistId=channel_upload_playlist,
        maxResults=count_video,
        # pageToken=next_page_token 
    )
    
    response_video = request_video.execute()

    # print(json.dumps(response_videos_id, indent=4))

    video_items = response_video['items'] 
    youtube_watch_url = 'https://www.youtube.com/watch?v='
    videos = { channel_name : [{'title':item['snippet']['title'], 
             'url':youtube_watch_url + item['snippet']['resourceId']['videoId']}
             for item in video_items] }


    # print(json.dumps(videos, indent=4))
    
    # Токен для получения следующих результатов
    # next_page_token = response_video['nextPageToken'])

    with open('videos_with_api.json', 'w') as f:
        json.dump(videos, f, indent=4)

if __name__ == "__main__":
    # channel_name = input('Enter youtube channel name: ')
    # count_video = input('Enter count of video which you need to find: ')
    channel_name = 'Davie504'
    get_videos_api(channel_name)

Проверим файл с видео *videos_with_api.json* после работы с апи:

In [ ]:
import json

with open('videos_with_api.json', 'r') as f:
  videos = json.load(f)

print('Содержимое videos_with_api.json:\n', json.dumps(videos, indent=4))

Содержимое videos_with_api.json:
 {
    "Davie504": [
        {
            "title": "what are they doing",
            "url": "https://www.youtube.com/watch?v=bCZbh1gETVw"
        },
        {
            "title": "this is forbidden",
            "url": "https://www.youtube.com/watch?v=udgWAaaMQpM"
        },
        {
            "title": "200 BASS GUITARS, 1 SOLO (World Record)",
            "url": "https://www.youtube.com/watch?v=mjd0FMOE0t8"
        },
        {
            "title": "BASSta's Paradise (Official Video)",
            "url": "https://www.youtube.com/watch?v=4DVy-5cOmlQ"
        },
        {
            "title": "The Ugly Truth About This 69 Strings Bass..",
            "url": "https://www.youtube.com/watch?v=qvxy-xW9dKo"
        }
    ]
}


# ***Парсер Youtube без API***

Установка на облако *selenium*

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 32.2 MB/s 
     |████████████████████████████████| 384 kB 54.0 MB/s 
     |████████████████████████████████| 140 kB 46.6 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
I

Работа с *selenium* и получение сначала плейлиста со всеми видео, а потом самого списка видео.

**Внимание!** *Google Colab* периодически работает **некорректно** с *Selenium* и не может подключиться к сайту. На локальном компьютере всё отрабатывает корректно.

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup as BS
from datetime import datetime
import json
import time

def get_videos_without_api(channel_name, count_video=5):
    videos = { channel_name : [] }
    url = f'https://www.youtube.com/c/{channel_name}'
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--window-size=1420,1080')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)
    timeout = 10
    try:
        element_present = EC.presence_of_element_located((By.TAG_NAME, 'body'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Превышен лимит загрузки страницы")
        return

    time.sleep(3)

    html = driver.page_source
    try:
        # Ищем id плейлиста со всеми видео
        soup = BS(html, 'lxml')

        # Первая кнопка проигрыша видео содежит в себе id плейлиста всех видео
        play_button = soup.find('div', id='play-button', class_='style-scope ytd-shelf-renderer')
        item = play_button.find('a', class_='yt-simple-endpoint style-scope ytd-button-renderer')
        
        playlist = item.get('href')
        if (not playlist):
            raise Exception('js скрипт не загрузился за отведённое время')
        url_playlist = f'https://www.youtube.com{playlist}'

        driver.get(url_playlist)
        try:
            element_present = EC.presence_of_element_located((By.TAG_NAME, 'body'))
            WebDriverWait(driver, timeout).until(element_present)
        except TimeoutException:
            print("Превышен лимит загрузки страницы")
            return
        time.sleep(3)

        html = driver.page_source
        soup = BS(html, 'lxml')
        videos_tags_list = soup.find_all('ytd-playlist-panel-video-renderer', class_='style-scope ytd-playlist-panel-renderer')

        for i in range(count_video):
            url = videos_tags_list[i].find('a', class_='yt-simple-endpoint style-scope ytd-playlist-panel-video-renderer', id='wc-endpoint').get('href')
            title = videos_tags_list[i].find('span', class_='style-scope ytd-playlist-panel-video-renderer', id='video-title').get('title')
            videos[channel_name].append({'title':title, 'url':f'https://www.youtube.com{url}'})

    except Exception as e:
        print('Parser error:', e)
    finally:
        driver.quit()
    
    with open('videos_without_api.json', 'w') as f:
        json.dump(videos, f, indent=4)

if __name__ == "__main__":
    # channel_name = input('Enter youtube channel name: ')
    # count_video = input('Enter count of video which you need to find: ')
    channel_name = 'Davie504'
    get_videos_without_api(channel_name)

Проверим файл с видео *videos_without_api.json* после работы с апи:

In [ ]:
import json

with open('videos_without_api.json', 'r') as f:
  videos = json.load(f)

print('Содержимое videos_without_api.json:\n', json.dumps(videos, indent=4))

Содержимое videos_without_api.json:
 {
    "Davie504": [
        {
            "title": "what are they doing",
            "url": "https://www.youtube.com/watch?v=bCZbh1gETVw&list=UUgFvT6pUq9HLOvKBYERzXSQ&index=1"
        },
        {
            "title": "this is forbidden",
            "url": "https://www.youtube.com/watch?v=udgWAaaMQpM&list=UUgFvT6pUq9HLOvKBYERzXSQ&index=2"
        },
        {
            "title": "200 BASS GUITARS, 1 SOLO (World Record)",
            "url": "https://www.youtube.com/watch?v=mjd0FMOE0t8&list=UUgFvT6pUq9HLOvKBYERzXSQ&index=3"
        },
        {
            "title": "BASSta's Paradise (Official Video)",
            "url": "https://www.youtube.com/watch?v=4DVy-5cOmlQ&list=UUgFvT6pUq9HLOvKBYERzXSQ&index=4"
        },
        {
            "title": "The Ugly Truth About This 69 Strings Bass..",
            "url": "https://www.youtube.com/watch?v=qvxy-xW9dKo&list=UUgFvT6pUq9HLOvKBYERzXSQ&index=5"
        }
    ]
}
